In [1]:
import pandas as pd
import numpy as np

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
csv_file_1 = "../ETL_Project_Corona/2019_nCoV_data.csv"
corona_data_df = pd.read_csv(csv_file_1)
corona_data_df.head()

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,1/22/2020 12:00,Anhui,China,1/22/2020 12:00,1,0,0
1,2,1/22/2020 12:00,Beijing,China,1/22/2020 12:00,14,0,0
2,3,1/22/2020 12:00,Chongqing,China,1/22/2020 12:00,6,0,0
3,4,1/22/2020 12:00,Fujian,China,1/22/2020 12:00,1,0,0
4,5,1/22/2020 12:00,Gansu,China,1/22/2020 12:00,0,0,0


In [4]:
death_sum= corona_data_df['Deaths'].sum()
death_sum

7148

In [5]:
confirmed_sum= corona_data_df['Confirmed'].sum()
confirmed_sum

331180

In [6]:
mortality_rate= (((death_sum/confirmed_sum)*100).round(decimals=2)).astype(str) + '%'
mortality_rate

'2.16%'

In [7]:
# corona_data_df = corona_data_df.rename(columns={"Province/State": "Province_State"})
# corona_data_df

In [8]:
countries =  corona_data_df['Country'].unique()
countries

array(['China', 'US', 'Japan', 'Thailand', 'South Korea',
       'Mainland China', 'Hong Kong', 'Macau', 'Taiwan', 'Singapore',
       'Philippines', 'Malaysia', 'Vietnam', 'Australia', 'Mexico',
       'Brazil', 'France', 'Nepal', 'Canada', 'Cambodia', 'Sri Lanka',
       'Ivory Coast', 'Germany', 'Finland', 'United Arab Emirates',
       'India', 'Italy', 'Sweden', 'Russia', 'Spain', 'UK', 'Belgium',
       'Others'], dtype=object)

In [9]:
Province_State = corona_data_df['Province/State'].unique()
Province_State

array(['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong',
       'Guangxi', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan',
       'Hong Kong', 'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu',
       'Jiangxi', 'Jilin', 'Liaoning', 'Macau', 'Ningxia', 'Qinghai',
       'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi', 'Sichuan', 'Taiwan',
       'Tianjin', 'Tibet', 'Washington', 'Xinjiang', 'Yunnan', 'Zhejiang',
       nan, 'Chicago', 'Illinois', 'California', 'Arizona', 'Ontario',
       'New South Wales', 'Victoria', 'Bavaria', 'British Columbia',
       'Queensland', 'Chicago, IL', 'South Australia', 'Boston, MA',
       'Los Angeles, CA', 'Orange, CA', 'Santa Clara, CA', 'Seattle, WA',
       'Tempe, AZ', 'Toronto, ON', 'San Benito, CA', 'London, ON',
       'Madison, WI', 'Cruise Ship', 'Diamond Princess cruise ship'],
      dtype=object)

In [10]:
country_df= corona_data_df.groupby([ 'Country'])
country_df.head()

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,1/22/2020 12:00,Anhui,China,1/22/2020 12:00,1,0,0
1,2,1/22/2020 12:00,Beijing,China,1/22/2020 12:00,14,0,0
2,3,1/22/2020 12:00,Chongqing,China,1/22/2020 12:00,6,0,0
3,4,1/22/2020 12:00,Fujian,China,1/22/2020 12:00,1,0,0
4,5,1/22/2020 12:00,Gansu,China,1/22/2020 12:00,0,0,0
...,...,...,...,...,...,...,...,...
968,969,2/7/2020 20:24,NaN,Belgium,4/2/2020 15:43,1,0,0
1009,1010,2/8/2020 23:04,Cruise Ship,Others,7/2/2020 16:33,61,0,0
1040,1041,2/8/2020 23:04,NaN,Belgium,4/2/2020 15:43,1,0,0
1081,1082,2/9/2020 23:20,Diamond Princess cruise ship,Others,2/9/2020 6:13,64,0,0


In [11]:
#Confirmed to death to recovery ratio, you can deduced the results are based the availibility/ quality of medical facilities. 
#Mainland china does not have enough facilities and personel to treat all confirmed cases. Higher death rates result.
total_cdr_country_df = country_df["Confirmed","Deaths","Recovered"].sum()
total_cdr_country_df.head(37)

,Confirmed,Deaths,Recovered
Country,,,
Australia,179,0,24
Belgium,7,0,0
Brazil,0,0,0
Cambodia,15,0,0
Canada,66,0,0
China,549,0,0
Finland,13,0,0
France,106,0,0
Germany,140,0,0


In [12]:
total_cdr_country_df.insert(0, 'Id', range(1, 1 + len(total_cdr_country_df)))

In [13]:
total_cdr_country_df = total_cdr_country_df.set_index("Id")
total_cdr_country_df.head()

,Confirmed,Deaths,Recovered
Id,,,
1,179,0,24
2,7,0,0
3,0,0,0
4,15,0,0
5,66,0,0


In [14]:
# total_cdr_country_df= total_cdr_country_df.add_column(0,'Country', corona_data_df['Country'])
# total_cdr_country_df
country=['Australia', 'Belgium', 'Brazil', 'Cambodia', 'Canada',
       'China', 'Finland', 'France', 'Germany', 'Hong Kong',
       'India', 'Italy', 'Ivory Coast', 'Japan', 'Macau',
       'Mainland China', 'Malaysia', 'Mexico', 'Nepal', 'Others', 'Philippines',
       'Russia', 'Singapore', 'South Korea', 'Spain',
       'Sri Lanka', 'Sweden','Taiwan', 'Thailand', 'UK', 'US', 'United Arab Emirates', 'Vietnam']

In [15]:
confirmed_sum = country_df["Confirmed"].sum()
deaths_sum = country_df["Deaths"].sum()
recovered_sum = country_df["Recovered"].sum()


In [16]:
# total_cdr_country_df['Country'] = country
# country_totals_df= total_cdr_country_df
# country_totals_df
total_country_df= pd.DataFrame({"Country":  country,
                                          
                                    "Confirmed_Total": confirmed_sum,
                                    "Death_Total":  deaths_sum,
                                    "Recovered_Total": recovered_sum
                                        
})
total_country_df

,Country,Confirmed_Total,Death_Total,Recovered_Total
Country,,,,
Australia,Australia,179,0,24
Belgium,Belgium,7,0,0
Brazil,Brazil,0,0,0
Cambodia,Cambodia,15,0,0
Canada,Canada,66,0,0
China,China,549,0,0
Finland,Finland,13,0,0
France,France,106,0,0
Germany,Germany,140,0,0


In [17]:
total_country_df.insert(0, 'Id', range(1, 1 + len(country)))
total_country_df = total_country_df.set_index("Id")
total_country_df

,Country,Confirmed_Total,Death_Total,Recovered_Total
Id,,,,
1,Australia,179,0,24
2,Belgium,7,0,0
3,Brazil,0,0,0
4,Cambodia,15,0,0
5,Canada,66,0,0
6,China,549,0,0
7,Finland,13,0,0
8,France,106,0,0
9,Germany,140,0,0


In [18]:
c_country_df = country_df["Confirmed"].sum()
c_country_df 

Country
Australia                  179
Belgium                      7
Brazil                       0
Cambodia                    15
Canada                      66
China                      549
Finland                     13
France                     106
Germany                    140
Hong Kong                  300
India                       29
Italy                       28
Ivory Coast                  0
Japan                      312
Macau                      139
Mainland China          327215
Malaysia                   163
Mexico                       0
Nepal                       17
Others                     321
Philippines                 26
Russia                      22
Singapore                  351
South Korea                245
Spain                       13
Sri Lanka                   15
Sweden                      11
Taiwan                     188
Thailand                   347
UK                          31
US                         149
United Arab Emirates        68


In [19]:
d_country_df = country_df["Deaths"].sum()
d_country_df 

Country
Australia                  0
Belgium                    0
Brazil                     0
Cambodia                   0
Canada                     0
China                      0
Finland                    0
France                     0
Germany                    0
Hong Kong                  7
India                      0
Italy                      0
Ivory Coast                0
Japan                      0
Macau                      0
Mainland China          7131
Malaysia                   0
Mexico                     0
Nepal                      0
Others                     0
Philippines               10
Russia                     0
Singapore                  0
South Korea                0
Spain                      0
Sri Lanka                  0
Sweden                     0
Taiwan                     0
Thailand                   0
UK                         0
US                         0
United Arab Emirates       0
Vietnam                    0
Name: Deaths, dtype: int64

In [20]:
r_country_df = country_df["Recovered"].sum()
r_country_df 

Country
Australia                  24
Belgium                     0
Brazil                      0
Cambodia                    0
Canada                      0
China                       0
Finland                     0
France                      0
Germany                     0
Hong Kong                   0
India                       0
Italy                       0
Ivory Coast                 0
Japan                      19
Macau                       5
Mainland China          17343
Malaysia                    4
Mexico                      0
Nepal                       0
Others                      0
Philippines                 0
Russia                      0
Singapore                   6
South Korea                10
Spain                       0
Sri Lanka                   3
Sweden                      0
Taiwan                      5
Thailand                   91
UK                          0
US                          6
United Arab Emirates        0
Vietnam                    10
Na

In [21]:
death_percentages= ((d_country_df/c_country_df)*100).round(decimals=2).astype(str) + '%'

death_percentages

Country
Australia                 0.0%
Belgium                   0.0%
Brazil                    nan%
Cambodia                  0.0%
Canada                    0.0%
China                     0.0%
Finland                   0.0%
France                    0.0%
Germany                   0.0%
Hong Kong                2.33%
India                     0.0%
Italy                     0.0%
Ivory Coast               nan%
Japan                     0.0%
Macau                     0.0%
Mainland China           2.18%
Malaysia                  0.0%
Mexico                    nan%
Nepal                     0.0%
Others                    0.0%
Philippines             38.46%
Russia                    0.0%
Singapore                 0.0%
South Korea               0.0%
Spain                     0.0%
Sri Lanka                 0.0%
Sweden                    0.0%
Taiwan                    0.0%
Thailand                  0.0%
UK                        0.0%
US                        0.0%
United Arab Emirates      0.0%


In [22]:
recovered_percentages= ((r_country_df/c_country_df)*100).round(decimals=2).astype(str) + '%'

recovered_percentages

Country
Australia               13.41%
Belgium                   0.0%
Brazil                    nan%
Cambodia                  0.0%
Canada                    0.0%
China                     0.0%
Finland                   0.0%
France                    0.0%
Germany                   0.0%
Hong Kong                 0.0%
India                     0.0%
Italy                     0.0%
Ivory Coast               nan%
Japan                    6.09%
Macau                     3.6%
Mainland China            5.3%
Malaysia                 2.45%
Mexico                    nan%
Nepal                     0.0%
Others                    0.0%
Philippines               0.0%
Russia                    0.0%
Singapore                1.71%
South Korea              4.08%
Spain                     0.0%
Sri Lanka                20.0%
Sweden                    0.0%
Taiwan                   2.66%
Thailand                26.22%
UK                        0.0%
US                       4.03%
United Arab Emirates      0.0%


In [23]:
percentages_country_df= pd.DataFrame({"Deaths_Percentage": death_percentages,
                                          
                                    "Recovered_Percentage": recovered_percentages,
                                    "Country":  country
                                        
})
percentages_country_df

,Deaths_Percentage,Recovered_Percentage,Country
Country,,,
Australia,0.0%,13.41%,Australia
Belgium,0.0%,0.0%,Belgium
Brazil,nan%,nan%,Brazil
Cambodia,0.0%,0.0%,Cambodia
Canada,0.0%,0.0%,Canada
China,0.0%,0.0%,China
Finland,0.0%,0.0%,Finland
France,0.0%,0.0%,France
Germany,0.0%,0.0%,Germany


In [24]:
percentages_country_df.insert(0, 'Id', range(1, 1 + len(country)))
percentages_country_df

,Id,Deaths_Percentage,Recovered_Percentage,Country
Country,,,,
Australia,1,0.0%,13.41%,Australia
Belgium,2,0.0%,0.0%,Belgium
Brazil,3,nan%,nan%,Brazil
Cambodia,4,0.0%,0.0%,Cambodia
Canada,5,0.0%,0.0%,Canada
China,6,0.0%,0.0%,China
Finland,7,0.0%,0.0%,Finland
France,8,0.0%,0.0%,France
Germany,9,0.0%,0.0%,Germany


In [25]:
percentages_country_df =percentages_country_df.set_index("Id")
percentages_country_df

,Deaths_Percentage,Recovered_Percentage,Country
Id,,,
1,0.0%,13.41%,Australia
2,0.0%,0.0%,Belgium
3,nan%,nan%,Brazil
4,0.0%,0.0%,Cambodia
5,0.0%,0.0%,Canada
6,0.0%,0.0%,China
7,0.0%,0.0%,Finland
8,0.0%,0.0%,France
9,0.0%,0.0%,Germany


In [26]:
new_corona_data_df = corona_data_df[['Sno','Last Update', 'Country','Province/State','Confirmed','Deaths','Recovered']].copy()
new_corona_data_df

,Sno,Last Update,Country,Province/State,Confirmed,Deaths,Recovered
0,1,1/22/2020 12:00,China,Anhui,1,0,0
1,2,1/22/2020 12:00,China,Beijing,14,0,0
2,3,1/22/2020 12:00,China,Chongqing,6,0,0
3,4,1/22/2020 12:00,China,Fujian,1,0,0
4,5,1/22/2020 12:00,China,Gansu,0,0,0
...,...,...,...,...,...,...,...
1194,1195,2/1/2020 19:53,US,"Los Angeles, CA",1,0,0
1195,1196,2/5/2020 21:53,US,"Madison, WI",1,0,0
1196,1197,2/1/2020 19:53,US,"Orange, CA",1,0,0
1197,1198,2/9/2020 7:03,US,"Seattle, WA",1,0,1


In [27]:
new_corona_data_df = new_corona_data_df.rename(columns={"Sno": "Id"})
new_corona_data_df 

,Id,Last Update,Country,Province/State,Confirmed,Deaths,Recovered
0,1,1/22/2020 12:00,China,Anhui,1,0,0
1,2,1/22/2020 12:00,China,Beijing,14,0,0
2,3,1/22/2020 12:00,China,Chongqing,6,0,0
3,4,1/22/2020 12:00,China,Fujian,1,0,0
4,5,1/22/2020 12:00,China,Gansu,0,0,0
...,...,...,...,...,...,...,...
1194,1195,2/1/2020 19:53,US,"Los Angeles, CA",1,0,0
1195,1196,2/5/2020 21:53,US,"Madison, WI",1,0,0
1196,1197,2/1/2020 19:53,US,"Orange, CA",1,0,0
1197,1198,2/9/2020 7:03,US,"Seattle, WA",1,0,1


In [28]:
# new_corona_data_df = new_corona_data_df.rename(columns={"Province/State": "Province_State"})

In [29]:
new_corona_data_df.set_index("Id")
new_corona_data_df = new_corona_data_df.set_index("Id")
new_corona_data_df.head()

,Last Update,Country,Province/State,Confirmed,Deaths,Recovered
Id,,,,,,
1,1/22/2020 12:00,China,Anhui,1,0,0
2,1/22/2020 12:00,China,Beijing,14,0,0
3,1/22/2020 12:00,China,Chongqing,6,0,0
4,1/22/2020 12:00,China,Fujian,1,0,0
5,1/22/2020 12:00,China,Gansu,0,0,0


In [30]:
csv_file_2 = "../ETL_Project_Corona/time_series_2019_ncov_confirmed.csv"
confirmed_data_df = pd.read_csv(csv_file_2)
confirmed_data_df.head()

,Province/State,Country/Region,Lat,Long,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 22:04,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,1.0,9.0,15.0,15.0,39.0,...,530.0,591.0,591.0,591.0,665,733,733,779,779,830
1,Beijing,Mainland China,40.18238,116.4142,10.0,14.0,22.0,26.0,36.0,36.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
2,Chongqing,Mainland China,30.05718,107.8740,5.0,6.0,9.0,27.0,27.0,57.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
3,Fujian,Mainland China,26.07783,117.9895,NaN,1.0,5.0,5.0,10.0,10.0,...,205.0,215.0,215.0,215.0,224,239,239,250,250,261
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,2.0,2.0,2.0,4.0,...,57.0,62.0,62.0,62.0,67,71,79,79,79,83


In [31]:
confirmed_data_df = confirmed_data_df.drop(['Lat','Long'], axis=1)

In [32]:
confirmed_data_df.insert(0, 'Id', range(1, 1 + len(confirmed_data_df)))

In [33]:
confirmed_data_df = confirmed_data_df.set_index("Id")

In [34]:
# confirmed_data_df = confirmed_data_df.rename(columns={"Country/Region": "Country"})

In [35]:
# confirmed_data_df = confirmed_data_df.rename(columns={"Province/State": "Province_State"})
confirmed_data_df

,Province/State,Country/Region,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 22:04,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
Id,,,,,,,,,,,,,,,,,,,,,
1,Anhui,Mainland China,NaN,1.0,9.0,15.0,15.0,39.0,39.0,60.0,...,530.0,591.0,591.0,591.0,665,733,733,779,779,830
2,Beijing,Mainland China,10.0,14.0,22.0,26.0,36.0,36.0,41.0,51.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
3,Chongqing,Mainland China,5.0,6.0,9.0,27.0,27.0,57.0,57.0,75.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
4,Fujian,Mainland China,NaN,1.0,5.0,5.0,10.0,10.0,18.0,18.0,...,205.0,215.0,215.0,215.0,224,239,239,250,250,261
5,Gansu,Mainland China,NaN,NaN,2.0,2.0,2.0,4.0,4.0,7.0,...,57.0,62.0,62.0,62.0,67,71,79,79,79,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"Boston, MA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
69,"San Benito, CA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2,2,2,2,2,2
70,NaN,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1


In [36]:
csv_file_3 = "../ETL_Project_Corona/time_series_2019_ncov_deaths.csv"
deaths_data_df = pd.read_csv(csv_file_3)
deaths_data_df.head()

,Province/State,Country/Region,Lat,Long,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 10:24,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
1,Beijing,Mainland China,40.18238,116.4142,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
2,Chongqing,Mainland China,30.05718,107.8740,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
3,Fujian,Mainland China,26.07783,117.9895,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2


In [37]:
deaths_data_df = deaths_data_df.drop(['Lat','Long'], axis=1)

In [38]:
deaths_data_df.insert(0, 'Id', range(1, 1 + len(deaths_data_df)))

In [39]:
deaths_data_df = deaths_data_df.set_index("Id")

In [40]:
# deaths_data_df = deaths_data_df.rename(columns={"Country/Region": "Country"})

In [41]:
# deaths_data_df = deaths_data_df.rename(columns={"Province/State": "Province_State"})
deaths_data_df

,Province/State,Country/Region,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 10:24,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
Id,,,,,,,,,,,,,,,,,,,,,
1,Anhui,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
2,Beijing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
3,Chongqing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
4,Fujian,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
5,Gansu,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"Boston, MA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
69,"San Benito, CA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
70,NaN,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [42]:
death_groupby = deaths_data_df.groupby([ 'Country/Region'])
death_groupby.head()

,Province/State,Country/Region,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 10:24,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
Id,,,,,,,,,,,,,,,,,,,,,
1,Anhui,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
2,Beijing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
3,Chongqing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
4,Fujian,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
5,Gansu,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2
32,Hong Kong,Hong Kong,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1,1,1,1
33,Macau,Macau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
34,Taiwan,Taiwan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
35,"Seattle, WA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [43]:
csv_file_4 = "../ETL_Project_Corona/time_series_2019_ncov_deaths.csv"
recovered_data_df = pd.read_csv(csv_file_4)
recovered_data_df.head()

,Province/State,Country/Region,Lat,Long,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 10:24,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
1,Beijing,Mainland China,40.18238,116.4142,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
2,Chongqing,Mainland China,30.05718,107.8740,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
3,Fujian,Mainland China,26.07783,117.9895,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2


In [44]:
recovered_data_df = recovered_data_df.drop(['Lat','Long'], axis=1)

In [45]:
recovered_data_df.insert(0, 'Id', range(1, 1 + len(recovered_data_df)))

In [46]:
recovered_data_df = recovered_data_df.set_index("Id")

In [47]:
# recovered_data_df = recovered_data_df.rename(columns={"Country/Region": "Country"})

In [48]:
# recovered_data_df = recovered_data_df.rename(columns={"Province/State": "Province_State"})
recovered_data_df

,Province/State,Country/Region,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 10:24,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
Id,,,,,,,,,,,,,,,,,,,,,
1,Anhui,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
2,Beijing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
3,Chongqing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
4,Fujian,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
5,Gansu,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"Boston, MA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
69,"San Benito, CA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
70,NaN,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [49]:
rds_connection_string = "postgres:postgres@localhost:5432/Corona_db"
engine = create_engine(f'postgresql://{rds_connection_string}')


In [50]:
total_country_df.to_sql(name='total_country', con=engine,  if_exists='append', index=False)

In [51]:
percentages_country_df.to_sql(name='country_percentages', con=engine,  if_exists='append', index=False)

In [52]:
new_corona_data_df.to_sql(name='corona_data', con=engine, if_exists='append', index=False)

In [53]:
confirmed_data_df.to_sql(name='confirmed_data', con=engine, if_exists='append', index=False)

In [54]:
deaths_data_df .to_sql(name='deaths_data', con=engine, if_exists='append', index=False)

In [55]:
recovered_data_df .to_sql(name='recovered_data', con=engine, if_exists='append', index=False)

In [56]:
pd.read_sql_query('select * from total_country', con=engine)

,Country,Confirmed_Total,Death_Total,Recovered_Total
0,Australia,179,0,24
1,Belgium,7,0,0
2,Brazil,0,0,0
3,Cambodia,15,0,0
4,Canada,66,0,0
...,...,...,...,...
61,Thailand,347,0,91
62,UK,31,0,0
63,US,149,0,6
64,United Arab Emirates,68,0,0


In [57]:
pd.read_sql_query('select * from country_percentages', con=engine)

,Deaths_Percentage,Recovered_Percentage,Country
0,0.0%,13.41%,Australia
1,0.0%,0.0%,Belgium
2,nan%,nan%,Brazil
3,0.0%,0.0%,Cambodia
4,0.0%,0.0%,Canada
...,...,...,...
61,0.0%,26.22%,Thailand
62,0.0%,0.0%,UK
63,0.0%,4.03%,US
64,0.0%,0.0%,United Arab Emirates


In [58]:
pd.read_sql_query('select * from corona_data', con=engine)

,Last Update,Country,Province/State,Confirmed,Deaths,Recovered
0,1/22/2020 12:00,China,Anhui,1,0,0
1,1/22/2020 12:00,China,Beijing,14,0,0
2,1/22/2020 12:00,China,Chongqing,6,0,0
3,1/22/2020 12:00,China,Fujian,1,0,0
4,1/22/2020 12:00,China,Gansu,0,0,0
...,...,...,...,...,...,...
2393,2/1/2020 19:53,US,"Los Angeles, CA",1,0,0
2394,2/5/2020 21:53,US,"Madison, WI",1,0,0
2395,2/1/2020 19:53,US,"Orange, CA",1,0,0
2396,2/9/2020 7:03,US,"Seattle, WA",1,0,1


In [59]:
pd.read_sql_query('select * from confirmed_data', con=engine)

,Province/State,Country/Region,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 22:04,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
0,Anhui,Mainland China,NaN,1.0,9.0,15.0,15.0,39.0,39.0,60.0,...,530.0,591.0,591.0,591.0,665,733,733,779,779,830
1,Beijing,Mainland China,10.0,14.0,22.0,26.0,36.0,36.0,41.0,51.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
2,Chongqing,Mainland China,5.0,6.0,9.0,27.0,27.0,57.0,57.0,75.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
3,Fujian,Mainland China,NaN,1.0,5.0,5.0,10.0,10.0,18.0,18.0,...,205.0,215.0,215.0,215.0,224,239,239,250,250,261
4,Gansu,Mainland China,NaN,NaN,2.0,2.0,2.0,4.0,4.0,7.0,...,57.0,62.0,62.0,62.0,67,71,79,79,79,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,None,Spain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,2
140,South Australia,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2,2,2,2,2,2
141,"Boston, MA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
142,"Madison, WI",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,1,1,1,1,1,1


In [60]:
pd.read_sql_query('select * from deaths_data', con=engine)

,Province/State,Country/Region,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 10:24,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
0,Anhui,Mainland China,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
1,Beijing,Mainland China,None,None,None,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
2,Chongqing,Mainland China,None,None,None,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
3,Fujian,Mainland China,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,Gansu,Mainland China,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,"Boston, MA",US,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
140,"San Benito, CA",US,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
141,None,Belgium,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
142,"Madison, WI",US,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [61]:
pd.read_sql_query('select * from recovered_data', con=engine)

,Province/State,Country/Region,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 10:24,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
0,Anhui,Mainland China,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
1,Beijing,Mainland China,None,None,None,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
2,Chongqing,Mainland China,None,None,None,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
3,Fujian,Mainland China,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,Gansu,Mainland China,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,"Boston, MA",US,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
140,"San Benito, CA",US,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
141,None,Belgium,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
142,"Madison, WI",US,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [62]:
engine.table_names()

['deaths_data',
 'total_country',
 'country_percentages',
 'corona_data',
 'confirmed_data',
 'recovered_data']